# Sentiment Analysis

In [17]:
import pandas as pd

In [19]:
twit=pd.read_csv('twitter_validation1 (2).csv',header=None)


In [21]:
twit.head()

,0,1,2,3
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


In [23]:
twit.columns=['ID','Social Media','Target','Text']
twit

,ID,Social Media,Target,Text
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...
...,...,...,...,...
995,4891,GrandTheftAuto(GTA),Irrelevant,⭐️ Toronto is the arts and culture capital of ...
996,4359,CS-GO,Irrelevant,tHIS IS ACTUALLY A GOOD MOVE TOT BRING MORE VI...
997,2652,Borderlands,Positive,Today sucked so it’s time to drink wine n play...
998,8069,Microsoft,Positive,Bought a fraction of Microsoft today. Small wins.


In [24]:
twit.drop(twit.index[(twit['Target']=='Irrelevant')],axis=0,inplace=True)
twit.reset_index(drop=True,inplace=True)
twit

,ID,Social Media,Target,Text
0,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
2,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,4433,Google,Neutral,Now the President is slapping Americans in the...
4,6273,FIFA,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...
...,...,...,...,...
823,314,Amazon,Negative,Please explain how this is possible! How can t...
824,9701,PlayStation5(PS5),Positive,Good on Sony. As much as I want to see the new...
825,2652,Borderlands,Positive,Today sucked so it’s time to drink wine n play...
826,8069,Microsoft,Positive,Bought a fraction of Microsoft today. Small wins.


In [27]:
swit=twit.drop(['ID','Social Media'],axis=1)
swit

,Target,Text
0,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,Negative,@Microsoft Why do I pay for WORD when it funct...
2,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,Neutral,Now the President is slapping Americans in the...
4,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...
...,...,...
823,Negative,Please explain how this is possible! How can t...
824,Positive,Good on Sony. As much as I want to see the new...
825,Positive,Today sucked so it’s time to drink wine n play...
826,Positive,Bought a fraction of Microsoft today. Small wins.


In [29]:
# swit['Target'].replace('Positive',1,inplace=True)
# swit['Target'].replace('Neutral',0,inplace=True)
# swit['Target'].replace('Negative',-1,inplace=True)
# swit['Target']=twit['Target'].astype(int)
# swit

In [31]:
# Assuming 'Target' contains values like 'Neutral', 'Negative', etc.
target_mapping = {'Positive':1,'Neutral': 0, 'Negative': -1}
swit['Target'] = swit['Target'].map(target_mapping).astype(int)
swit

,Target,Text
0,0,BBC News - Amazon boss Jeff Bezos rejects clai...
1,-1,@Microsoft Why do I pay for WORD when it funct...
2,-1,"CSGO matchmaking is so full of closet hacking,..."
3,0,Now the President is slapping Americans in the...
4,-1,Hi @EAHelp I’ve had Madeleine McCann in my cel...
...,...,...
823,-1,Please explain how this is possible! How can t...
824,1,Good on Sony. As much as I want to see the new...
825,1,Today sucked so it’s time to drink wine n play...
826,1,Bought a fraction of Microsoft today. Small wins.


In [33]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

wordnet=WordNetLemmatizer()
corpus=[]
for i in range(0, len(swit)):
    review=re.sub('[^a-zA-Z]',' ',swit['Text'][i])
    review=review.lower()
    review=nltk.word_tokenize(review)
    review=[wordnet.lemmatize(word) for word in review if not word in set(stopwords.words('english')) ]
    review=' '.join(review)
    corpus.append(review)

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=2500)
x=cv.fit_transform(corpus).toarray()
y=swit.Target

In [37]:
# train test split
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=.2,random_state=0)
# naive bayes classifier
from sklearn.naive_bayes import MultinomialNB
spam=MultinomialNB().fit(xtrain,ytrain)

In [39]:
y_pred=spam.predict(xtest)

In [41]:
y_pred

array([ 1,  0, -1,  0,  1,  1, -1, -1, -1,  1,  0, -1, -1, -1,  1,  1, -1,
        0,  1,  0,  1,  1,  1,  1,  1,  1, -1,  0,  1,  0,  1,  1, -1, -1,
       -1,  0, -1,  0, -1, -1,  1,  1,  1,  1,  0,  1, -1,  0, -1, -1,  1,
        0, -1,  1,  1, -1,  0, -1,  1, -1,  1,  1,  1, -1,  0,  1,  1,  0,
        1,  1,  1, -1,  1,  0,  1,  0, -1,  1,  0,  0,  0,  1,  0,  0,  0,
        1,  0,  0,  1, -1, -1,  1,  1,  1, -1,  1,  0,  1,  1, -1,  1,  0,
       -1,  1,  1,  0,  0,  1,  1,  1,  0,  0,  1,  1,  1, -1,  0,  1, -1,
       -1,  1,  1,  0,  1,  0,  1,  1,  0, -1,  1,  1,  1,  1,  0,  0,  1,
        0,  0,  0,  1,  1, -1,  1,  0, -1,  1,  0,  1, -1,  1, -1, -1,  0,
        1,  1,  1,  0,  1, -1,  0, -1, -1, -1,  1,  1,  0])

In [45]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(ytest,y_pred)

In [47]:
cm

array([[25,  9, 16],
       [11, 27, 18],
       [ 7, 10, 43]], dtype=int64)

In [49]:
spam.score(xtest,ytest)

0.572289156626506